In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import random
import time
import re
import json
import os.path
from os import path

In [ ]:
driver = webdriver.Chrome(executable_path='C:\chromedriver\chromedriver.exe')

In [ ]:
# Get number of matches from HLTV given full content and 1 star fitler
driver.get('https://www.hltv.org/results?content=stats&stars=1')
content = driver.page_source
soup = BeautifulSoup(content)
n = int(soup.find('span', {'class':'pagination-data'}).text.split()[-1])
offset = range(0, n, 100)
print(n)

In [ ]:
if not path.exists('matchIDs.csv'):

    matchIDs = pd.DataFrame(columns=['Date', 'ID', 'Status'])

    for page in offset:

        driver.get('https://www.hltv.org/results?offset=' + str(page) + '&content=stats&stars=1')
        content = driver.page_source
        soup = BeautifulSoup(content)
        contentpage = soup.find('div', {'class':'results-holder allres'})

        for subset in contentpage.findAll('div', {'class':'results-sublist'}):
            date = subset.find('div', {'class':'standard-headline'}).text
            date = date[12:]

            for URL in subset.findAll('a', {'class':'a-reset'}):
                ID = URL['href'].split('/')[2]
                matchIDs = matchIDs.append({'Date':date, 'ID':int(ID), 'Status':0}, ignore_index = True)

        print(page)
        sleeptime = random.uniform(1, 2)
        time.sleep(sleeptime)

    matchIDs = matchIDs[::-1].reset_index(drop = True)
    matchIDs.to_csv('matchIDs.csv', index = False)

else:
    print('File already exists')
    matchIDs = pd.read_csv('matchIDs.csv')

In [ ]:
if path.exists('matchIDs.csv'):

    harvest = pd.DataFrame()

    for page in offset:

        driver.get('https://www.hltv.org/results?offset=' + str(page) + '&content=stats&stars=1')
        content = driver.page_source
        soup = BeautifulSoup(content)
        contentpage = soup.find('div', {'class':'results-holder allres'})
        batch = pd.DataFrame(columns=['Date', 'ID', 'Status'])

        for subset in contentpage.findAll('div', {'class':'results-sublist'}):
            date = subset.find('div', {'class':'standard-headline'}).text
            date = date[12:]

            for URL in subset.findAll('a', {'class':'a-reset'}):
                ID = URL['href'].split('/')[2]
                batch = batch.append({'Date':date, 'ID':int(ID), 'Status':0}, ignore_index = True)
        
        booler = batch['ID'].isin(matchIDs['ID'])
        batch = batch[~booler]
        if sum(booler) == 100:
            print('Break')
            break

        harvest = harvest.append(batch)
        print(page)
        sleeptime = random.uniform(1, 2)
        time.sleep(sleeptime)

    harvest = harvest[::-1]
    matchIDs = matchIDs.append(harvest).reset_index(drop = True)
    print(str(len(harvest)) + ' new matche(s) has/have been found and added')

else:
    print('File does not exist, initialize first')

In [ ]:
iterations = 0
maxiter = 100
data = {}
for row in matchIDs.itertuples():

    if row.Status == 1:
        continue

    driver.get('https://www.hltv.org/matches/' + str(row.ID) +'/anything')
    content = driver.page_source
    soup = BeautifulSoup(content)

    matchdata  = {}
    matchdata['Date'] = row.Date
    matchdata['Event'] = soup.find('div', {'class':'event text-ellipsis'}).text

    team1info = soup.find('div', {'class': 'team1-gradient'})
    team1name = team1info.find('div', {'class':'teamName'}).text
    team1score = team1info.find('div', {'class':['won', 'lost', 'tie']}).text

    team2info = soup.find('div', {'class': 'team2-gradient'})
    team2name = team2info.find('div', {'class':'teamName'}).text
    team2score = team2info.find('div', {'class':['won', 'lost', 'tie']}).text

    teams = [team1name, team2name]
    score = [team1score, team2score]
    matchdata['Teams'] = teams
    matchdata['Score'] = score

    maps = []
    mapinfo = soup.find('div', {'class':'flexbox nowrap'})
    for m in mapinfo.findAll('div', {'class':'dynamic-map-name-full'})[1:]:
        maps.append(m.text)
    matchdata['Maps'] = maps

    try:
        oddsinfo = soup.find('div', {'class':'past-matches-grid'})
        odds = oddsinfo.findAll('div', {'class':'past-matches-bottom-right-numbers'})
        odds = [odds[0].text, odds[1].text]
    except:
        odds = 'No odds available'
    
    matchdata['Odds'] = odds

    matchlinks = []
    for div in soup.findAll('div', {'class':'results-center-stats'}):
        for a in div.findAll('a'):
            link = a['href']
            matchlinks.append(link)

    match = {}
    count = 0
    for link in matchlinks:

        driver.get('https://www.hltv.org/' + str(link))
        content = driver.page_source
        soup = BeautifulSoup(content)

        game_stats = {}
        scoreboard = soup.findAll('table', {'class':'stats-table'})
        team1 = pd.read_html(str(scoreboard))[0].values.tolist()
        team2 = pd.read_html(str(scoreboard))[1].values.tolist()
        scoreboard = {}
        scoreboard['Team 1'] = team1
        scoreboard['Team 2'] = team2
        game_stats['Scoreboard'] = scoreboard

        misc_stats = soup.findAll('div', attrs={'class': 'right'})
        game_stats['Starting side'] = misc_stats[0].contents[4]['class'][0][:-6]
        game_stats['Rounds'] = misc_stats[0].text
        game_stats['Team rating'] = misc_stats[1].text
        game_stats['Entries'] = misc_stats[2].text
        game_stats['Clutches'] = misc_stats[3].text

        count += 1
        match['Game ' + str(count)] = game_stats

        sleeptime = random.uniform(1, 2)
        time.sleep(sleeptime)

    matchdata['Match'] = match
    data[str(row.ID)] = matchdata
    matchIDs.loc[row.Index, 'Status'] = 1

    iterations +=1
    print(str(iterations) + ' out of ' + str(maxiter))

    if iterations == maxiter:
        print('Finished')
        break

if path.exists('main.json'):
    with open('main.json', 'r+') as f:
        main = json.load(f)
        main.update(data)
        f.seek(0)
        json.dump(main, f, indent = 1)
        f.close()
else:
    j = json.dumps(data, indent = 1)
    with open('main.json', 'w') as f:
        f.write(j)
        f.close()

matchIDs.to_csv('matchIDs.csv', index = False)

print(str(len(data)) + ' entries have been added')
print(str(len(matchIDs.Status) - sum(matchIDs.Status)) + ' entries awaiting to be scraped')

In [206]:
# Validate
test = json.load(open('main.json'))
len(test) == sum(matchIDs.Status)

True